# 5.i. BDR-specific querying

Int his module we will test SPARQL queries on the Biodiversity Data Repository (BDR) system.

Before training with this module, a revision of Modules 3 & 4 should be done first.

### Data Access

This module requires that queries be executed on a SPARQL Endpoint with access to BDR data. While public access to some demo BDR data is available, there may not be public access to the cull BDR content.

> **NOTE**: A BDR full content endpoint will be provisioned for this work, but it will not be recorded here for general, public, access.

## Outline

0. Revision
1. Test access
2. Understanding an RDF DB
    * generic approaches to discovering what's in any large RDF DB
3. Understanding the BDR
4. BDR Queries
    * mostly prepared by Ashley
5. Desktop GIS SPARQL Use

## 0. Revision

Review Module 4: the GeoSPARQL querying in particular.

## 1. Test access

We will just test access to the BDR's Read Only SPARQL endpoint, using the normal Python SPARQL client code we've used in other modules.

Part of the reason for doing this is to test out timeouts: the BDR contains lots of data and sometimes it will kill queries retreiving lots of information to prevent the system locking up.

In [ ]:
# set this with a real value
bdr_sparql_endpoint = "XXXX"

# import some Python packages
from kurra.sparql import query
from IPython.display import display, Markdown
from kurra.utils import render_sparql_result

# define our pretty-print function
def table_print(r):
    display(Markdown(render_sparql_result(r)))

In [ ]:
%%timeit -n 1 -r 1
# Force a timeout, maybe...? You will have to wait for this result

# This query should get a LOT of Concepts. The BDR might time out - 30s - 
# but still return all the results retrieved in that time.
#
# If there are problems, run this query in a SPARQL client
q = """
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT *
WHERE {
  ?o a skos:Concept .

  OPTIONAL {
    ?o skos:prefLabel ?label
  }
}
"""

r = query(bdr_sparql_endpoint, q)
table_print(r)  # at least 100k Concepts

In [ ]:
%%timeit -n 1 -r 1
# make a simple SPARQL query just to see things working and time it, using %%timeit above
q = """
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT *
WHERE {
  ?o a owl:Ontology .

  OPTIONAL {
    ?o skos:prefLabel ?label
  }
}
"""

r = query(bdr_sparql_endpoint, q)
table_print(r)

In [ ]:
%%timeit -n 1 -r 1
# Respect the timeout by using a limit
q = """
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT *
WHERE {
  ?o a skos:Concept .

  OPTIONAL {
    ?o skos:prefLabel ?label
  }
}
LIMIT 10
"""

r = query(bdr_sparql_endpoint, q)
table_print(r)

> **NOTE**: From now onwards, we will be applying queries to the BDR DB via a web-based SPARQL client.
>
> Many clients allow the underlying SPARQL endpoint to be altered as does this one that we will be using:
>
> https://demo.dev.kurrawong.ai/sparql

### 2. Understanding an RDF DB

To discover what's in an RDF DB when you know nothing at all about it, you can:

* Look for definitional items
* View the BDR structure

#### 2.1 Look for definitional items

* Search for any `owl:Ontology` instances
* Search for any `skos:ConceptScheme` instances

Re-use the first query above:

```sparql
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT *
WHERE {
  ?o a owl:Ontology .

  OPTIONAL {
    ?o skos:prefLabel ?label
  }
}
```

#### 2.2 View the BDR structure

* Search for Named Graphs

Named graphs don't _have_ to tell us anything about a DB - they may be arbitrarily identified - but they often do, by using IRIs that mean something.

```sparql
SELECT DISTINCT ?g
WHERE {
  GRAPH ?g {
   	?s ?p ?o
  }
}
LIMIT 100
```

Then try to `DESCRIBE` a graph. We will only get a result here if the IRI for a graph is also used in data. This _is_ the case with the BDR where Datasets are stored in graphs with the same IRI, so this `DESCRIBE` query for a graph will actually get Dataset metadata.

### 3. Understanding the BDR

#### Read the documentation

In this case, start at the [BDR Resources Page](https://resources.bdr.gov.au/):

* [ABIS](https://linked.data.gov.au/def/abis)
* [BDR Profile of ABIS](https://linked.data.gov.au/def/bdr-pr)
* [BDR Catalogues](https://resources.bdr.gov.au/catalogues)

[From ABIS](https://linked.data.gov.au/def/abis#annex-a), the main structure of data in the BDR is as follows:

<div>
    <img src="module-5i-files/rm-overview.svg" width="20%"/>
</div>

Note that the BDR contains a specialised kind of the `schema:Statement` classes, `abis:BiodiversityRecord`, and they are about many things but importantly `dwc:Occurrence` instances, so we will use the specialised form of the structure above:

(`schema:DataCatalog`)  
-- `schema:hasPart` -->  
(`schema:Dataset`)

(`schema:Dataset`)  
-- `schema:hasPart` -->  
(`abis:BiodiversityRecord`)

(`abis:BiodiversityRecord`)  
-- `schema:about` -->  
(`dwc:Occurrence`)

#### Ask someone

The BDR Team!

### 4. BDR Queries

#### 4.1 BDR Structure

The "main type of thing" in the BDR is occurrences, so...

##### 4.1.1 Count Occurrences

Count Occurrence.

```sparql
PREFIX dwc: <http://rs.tdwg.org/dwc/terms/>

SELECT (COUNT(?o) AS ?count)
WHERE {
	?o a dwc:Occurrence .
}
```

Counting queries are not always fun for RDF DBs, but this performs well.

##### 4.1.2 Find `Dataset` instances, and some metadata

```sparql
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX schema: <https://schema.org/>
SELECT ?ds ?ds_name ?provider ?provider_name 
WHERE {
    ?ds 
        a schema:Dataset ;
        schema:name ?ds_name ;
        prov:qualifiedAttribution [
          prov:hadRole <https://linked.data.gov.au/def/data-roles/resourceProvider> ;
          prov:agent ?provider ;
        ] ;
    .
    ?provider schema:name ?provider_name .
}
LIMIT 1000 # Limit because there are over 10,000 ds
```

##### 4.1.3 Count Records in a Dataset

<div>
    <img src="module-5i-files/Dataset_Records.svg" width="75%"/>
</div>

```sparql
PREFIX abis: <https://linked.data.gov.au/def/abis/>
PREFIX schema: <https://schema.org/>

SELECT ?ds (COUNT(?rec) as ?record_count)
WHERE {
    BIND (<https://linked.data.gov.au/dataset/bdr/63c890eb-cd66-4cb5-8518-45bd6dc1409c> as ?ds)
  
    ?rec
        a abis:BiodiversityRecord ;
        schema:isPartOf ?ds ;
    .
}
GROUP BY ?ds
```

##### 4.1.4 Count records in a set of Datasets

In every Dataset provided by QLD WildNet

(Find the Qld WildNet IRI by looking in the [BDR Organisations vocabulary](https://linked.data.gov.au/dataset/bdr/orgs)!)

```sparql
PREFIX abis: <https://linked.data.gov.au/def/abis/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX schema: <https://schema.org/>

SELECT ?dataset (COUNT(?rec) as ?record_count)
WHERE {
    ?dataset a schema:Dataset ;
    prov:qualifiedAttribution [
        prov:hadRole <https://linked.data.gov.au/def/data-roles/resourceProvider> ;
        prov:agent <https://linked.data.gov.au/org/qld-wildnet>
    ] .
    ?rec
        a abis:BiodiversityRecord ;
        schema:isPartOf ?dataset ;
    .
}
GROUP BY ?dataset
LIMIT 1000 # Must limit to 1000 because counting in graphs is SLOW!
```

#### 4.2 Finding an Occurrence

Occurrences are the main thing, not Records, so let's find some.

Find any 3 Occurrences in a given Dataset.

```sparql
PREFIX schema: <https://schema.org/>

SELECT ?ds ?occ
WHERE {

  BIND (<https://linked.data.gov.au/dataset/bdr/63c890eb-cd66-4cb5-8518-45bd6dc1409c> as ?ds)

  # inverse path from Dataset to Record and then from Record to Occurrence
  ?ds ^schema:isPartOf/schema:about ?occ .
}
LIMIT 3
```

Then `DESCRIBE` one of the results

```sparql
DESCRIBE <https://linked.data.gov.au/dataset/bdr/63c890eb-cd66-4cb5-8518-45bd6dc1409c/occurrence/6167022>
```

Notice the `schema:temporal` and `schema:spatial` predicates that indicate time and place.

The main properties of Occurrences other than time and place are indicated by `sosa:Observation` instances which observe the Occurrence and produce results. This is part of the normal "observarions & measurements" pattern we see in semantic representations of real-world sampling and observation, codified by the [SOSA Ontology within the Semantic Sensor Network Ontology](https://www.w3.org/TR/vocab-ssn/).

#### 4.3 Occurrence scenarios

Now we will find Occrrences, regardless of the Dataset they are in.

##### 4.3.1 Find Occurrences more recent than a date

```sparql
PREFIX dwc: <http://rs.tdwg.org/dwc/terms/>
PREFIX schema: <https://schema.org/>
PREFIX time: <http://www.w3.org/2006/time#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?occ ?date
WHERE {
  ?occ
    a dwc:Occurrence ;
	schema:temporal/time:inXSDDate ?date ;
  .

  FILTER (?date > "2010-01-01"^^xsd:date)
}
LIMIT 10
```

##### 4.3.2 Find Occurrences in an area

<div>
    <img src="module-5i-files/test_area.png" width="25%"/>
</div>

Area polygon:

```
POLYGON ((144.86776092396758 -37.217516909516405, 144.86776092396758 -37.445198608019595, 145.22280059904023 -37.445198608019595, 145.22280059904023 -37.217516909516405, 144.86776092396758 -37.217516909516405))
```

We just intersect, 'within' using that polygon and the location of Occurrences:

```sparql
PREFIX dwc: <http://rs.tdwg.org/dwc/terms/>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX geof: <http://www.opengis.net/def/function/geosparql/>
PREFIX schema: <https://schema.org/>

SELECT ?occ ?loc
WHERE {

  ?occ
    a dwc:Occurrence ;
	schema:spatial/geo:asWKT ?loc ;
  .

  FILTER geof:sfWithin(?loc, "POLYGON ((144.86776092396758 -37.217516909516405, 144.86776092396758 -37.445198608019595, 145.22280059904023 -37.445198608019595, 145.22280059904023 -37.217516909516405, 144.86776092396758 -37.217516909516405))"^^geo:wktLiteral)
}
LIMIT 10
```

##### 4.3.3 Find Occurrences later than a date and in an area

Combining a time and space filter:

```sparql
PREFIX dwc: <http://rs.tdwg.org/dwc/terms/>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX geof: <http://www.opengis.net/def/function/geosparql/>
PREFIX schema: <https://schema.org/>
PREFIX time: <http://www.w3.org/2006/time#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?occ ?loc ?date
WHERE {
  ?occ
    a dwc:Occurrence ;
	schema:temporal/time:inXSDDate ?date ;
	schema:spatial/geo:asWKT ?loc ;
  .

  FILTER (?date > "2010-01-01"^^xsd:date)


  FILTER geof:sfWithin(?loc, "POLYGON ((144.86776092396758 -37.217516909516405, 144.86776092396758 -37.445198608019595, 145.22280059904023 -37.445198608019595, 145.22280059904023 -37.217516909516405, 144.86776092396758 -37.217516909516405))"^^geo:wktLiteral)
}
LIMIT 10
```

Without the limit, the above query returns something like 650 Occurrences but times out. There are more Occurrences more recent that 2010 in the BDR for that area.

##### 4.3.4 Search by named area

Now we will use some (demo) reference data in the system to find Occurrences, rather than having to enter everything by hand.

For the dummy Cobaw State Forrest named area:

<div>
    <img src="module-5i-files/cobaw-sf.png" width="50%"/>
</div>

We have defined it in the BDR with the IRI `http://example.com/area/cobaw-sf` like this:

```rdf
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX schema: <https://schema.org/>

<http://example.com/area/cobaw-sf>
    a geo:Feature ;
    schema:name "Cobaw State Forrest" ;
    geo:hasGeometry [
        geo:asWKT "POLYGON ((144.58173016966896 -37.24956579600307, 144.5818620035476 -37.25705237162612, 144.58468167287208 -37.26184889584653, 144.59221483686815 -37.26942427042472, 144.60019930180073 -37.2696800509701, 144.6097569765136 -37.26589128698223, 144.62010279085024 -37.26487579225718, 144.62618657861077 -37.25755363983529, 144.6237288203106 -37.24923601040191, 144.63255784273662 -37.2456814431501, 144.6386422428355 -37.241658947642414, 144.64531452823854 -37.24014753547258, 144.65071046054516 -37.242169234416956, 144.63954421455082 -37.248714691887514, 144.63535893666818 -37.25173042604656, 144.63433036489675 -37.262588756430276, 144.6425708977398 -37.26384946139068, 144.6813580355166 -37.24992661329557, 144.67855378379767 -37.236301712712326, 144.70331906903363 -37.2312219790982, 144.7137851706711 -37.24737732142518, 144.72683071812446 -37.246618434745315, 144.77905750270378 -37.22091978048038, 144.78669844182093 -37.20900984721747, 144.78333652951426 -37.18805566664313, 144.77120910791263 -37.18733580190369, 144.76510374480932 -37.19707402051011, 144.7565106744849 -37.20240708467877, 144.73997664127745 -37.206205823232516, 144.7256687762963 -37.19936383204333, 144.72026345955146 -37.19201152148244, 144.72662802624728 -37.181833376218435, 144.72090071729718 -37.17500804799372, 144.70817585980274 -37.16763612998565, 144.6919636334743 -37.158323752619175, 144.68404140532044 -37.15235313072725, 144.66592364243428 -37.15410310168956, 144.63386541779158 -37.17390221756201, 144.63476735189215 -37.18700026896564, 144.64757539807601 -37.19117969282837, 144.65078089700654 -37.201332196460946, 144.64376598429857 -37.20836942900248, 144.63297681956277 -37.214184536051626, 144.6205829772585 -37.218217582676836, 144.6106379492657 -37.21688844229717, 144.58845707360524 -37.23669493906766, 144.58173016966896 -37.24956579600307))"^^geo:wktLiteral ;
    ] ;
.
```

So `DESCRIBE <http://example.com/area/cobaw-sf>` will return the above.

Since it is now in the system, we don't need to regurgitat out all its coordiantes to use it, we can just match them in a Graph Pattern Match, like this:

```sparql
PREFIX abis: <https://linked.data.gov.au/def/abis/>
PREFIX dwc: <http://rs.tdwg.org/dwc/terms/>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX geof: <http://www.opengis.net/def/function/geosparql/>
PREFIX schema: <https://schema.org/>

SELECT ?occ ?loc ?date
WHERE {
  # match Cobaw's coordinates
  <http://example.com/area/cobaw-sf>
    geo:hasGeometry/geo:asWKT ?cobaw_sf_loc ;
  .

  # match the coordinates of each Occurrence
  ?occ
    a dwc:Occurrence ;
	schema:spatial/geo:asWKT ?loc ;
  .

  # find all the Occurrences within Cobaw's area
  FILTER geof:sfWithin(?loc, ?cobaw_sf_loc)
}
LIMIT 10
```

##### 4.3.5 Coarse v. Fine geospatial searching

Searching with simpler polygons is easier for the BDR, so:

Find which Datasets' bounding boxes intersect with the bounding box of K'gari.

> NOTE: Datasets have footprints as bounding boxes that contain all their Occurrence's locations

<div>
    <img src="module-5i-files/kgari_box.png" width="50%"/>
</div>

```sparql
PREFIX geof: <http://www.opengis.net/def/function/geosparql/>
PREFIX schema: <https://schema.org/>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>

SELECT ?dataset
WHERE {
    BIND("""POLYGON ((152.935181 -25.787527, 153.382874 -25.787527, 153.382874 -24.684457, 152.935181 -24.684457, 152.935181 -25.787527))"""^^geo:wktLiteral AS ?Kgari_bbox)

    ?dataset
        a schema:Dataset ;
        geo:hasBoundingBox/geo:asWKT ?bounds
    .

    FILTER geof:sfIntersects(?bounds, ?Kgari_bbox)
}
LIMIT 1000
```

Now find which Datasets intersect with K'gari but use finer polygons for both the Dataset & K'gari.

<div>
    <img src="module-5i-files/kgari_hull.png" width="50%"/>
</div>

```sparql
PREFIX geof: <http://www.opengis.net/def/function/geosparql/>
PREFIX schema: <https://schema.org/>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>

SELECT ?dataset
WHERE {
    BIND("""POLYGON ((153.231812 -24.666986, 153.045044 -24.791722, 153.198853 -24.96614, 152.946167 -25.207426, 153.014832 -25.34899, 152.896729 -25.532528, 152.946167 -25.703413, 152.987366 -25.79, 153.099976 -25.819672, 153.388367 -25.000994, 153.316956 -24.66449, 153.231812 -24.666986))"""^^geo:wktLiteral as ?Kgari_polygon)

    ?dataset
        a schema:Dataset ;
        geo:hasGeometry/geo:asWKT ?bounds
    .

    FILTER geof:sfIntersects(?bounds, ?Kgari_polygon)
}
LIMIT 100
```

This query is much slower (but still only a few seconds).

### 4.4 National Species List Use

NSL relationships in the BDR (This is using the official published RDF version of the NSL)

<div>
    <img src="module-5i-files/NSL_Toplevel.svg" width="75%"/>
</div>


#### 4.4.1 Find which datasets contain a Species

_Velleia perfoliata R.Br._

![](NameUsageSubject.svg)

```sparql
PREFIX schema: <https://schema.org/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX nsl: <https://linked.data.gov.au/def/nsl/>
SELECT ?dataset
WHERE {
    {
        SELECT ?nsl_usage
        WHERE {
            GRAPH <https://biodiversity.org.au/nsl/2025-06-16> {
                ?nsl_name
                    a nsl:TaxonName ;
                    rdfs:label "Velleia perfoliata R.Br." ;
                .

                ?nsl_usage nsl:usageOf ?nsl_name .
            }
        }
    }

    ?dataset a schema:Dataset .

    ?nsl_usage
        schema:subjectOf ?dataset ;
    .
}
LIMIT 100
```

#### 4.4.2 Find location and date of all instances of Species

_Velleia perfoliata R.Br._, within one given Dataset:

Dataset: `https://linked.data.gov.au/dataset/bdr/c1130d9b-2655-4454-bab1-9c3cebe381dc`

```sparql
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX nsl: <https://linked.data.gov.au/def/nsl/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX schema: <https://schema.org/>
PREFIX time: <http://www.w3.org/2006/time#>

SELECT ?occurrence ?location ?datetime 
WHERE {
    {
        SELECT ?nsl_usage WHERE {
            GRAPH <https://biodiversity.org.au/nsl/2025-06-16> {
                ?nsl_name a nsl:TaxonName ;
                    rdfs:label "Velleia perfoliata R.Br.".
                ?nsl_usage nsl:usageOf ?nsl_name .
            }
        } 
    }
    BIND (<https://linked.data.gov.au/dataset/bdr/c1130d9b-2655-4454-bab1-9c3cebe381dc> as ?dataset)

    ?occurrence 
      	^schema:subjectOf ?nsl_usage ;
    	schema:spatial/geo:asWKT ?location ;
        schema:temporal/(time:inXSDDate|time:inXSDDateTime|time:inXSDDateTimeStamp|time:inXSDgYear|time:inXSDgYearMonth) ?datetime ;
    .
} 
LIMIT 10
```

### 4.5 Other aspects of Occurrences

Occurrences have Observations on them that establish other facts about them, other than time & space, such as the species (or at least name of a species) observed.

We see this pattern:

```sparql
?observation 
  sosa:hasFeatureOfInterest ?occurrence ;
  sosa:hasResult/rdf:value ?value
.
```

This Graph Pattern Match shows that Observations on Occurrences via `hasFeatureOfInterest` produce results with values that we want.

Here is an example:

<div>
    <img src="module-5i-files/ObservationsObservationCollections.svg" width="75%"/>
</div>


#### 4.5.1 Show all Observations on an Occurrence

We want the Observations, the names of the Observable Properties of the Occurrence they observed and the names of the Procedures they used, as well as the values of the Results.

These relations all follow SOSA's _Observation_ pattern, as per https://www.w3.org/TR/vocab-ssn/#x4-3-observations:

<div>
    <img src="module-5i-files/SOSA-Observation.png" width="75%"/>
</div>


```sparql
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX sosa: <http://www.w3.org/ns/sosa/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?observation ?property_label ?procedure_label ?result_value
WHERE {
    BIND (<https://linked.data.gov.au/dataset/bdr/63c890eb-cd66-4cb5-8518-45bd6dc1409c/occurrence/6790631> as ?occurrence)

    ?observation
        sosa:hasFeatureOfInterest ?occurrence ;
        sosa:observedProperty ?property ;
        sosa:usedProcedure ?procedure ;
        sosa:hasResult/rdf:value ?result_value ;
    .

    ?property skos:prefLabel ?property_label .
    ?procedure skos:prefLabel ?procedure_label .
}
```

#### 4.5.2

Find the scientific name for an Occurrence.

We do this by finding the Observation that targets the scientific name Observable Property, as defined by TERN in a vocab which can be ooked up online: 

**scientific name**  
<http://linked.data.gov.au/def/tern-cv/56195246-ec5d-4050-a1c6-af786fbec715>  
_The formal biological name, with authorship and date information, if known. When forming part of an identification, this should be the name in lowest level taxonomic rank that can be determined._

This is one of many Observable Properties that could be observed. The BDR contains observations for many Observable Properties defined by TERN in the [Parameter Types](http://linked.data.gov.au/def/tern-cv/5699eca7-9ef0-47a6-bcfb-9306e0e2b85e) and also other vocabularies.

```sparql
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX sosa: <http://www.w3.org/ns/sosa/>

SELECT ?r ?l
WHERE {
  BIND (<https://linked.data.gov.au/dataset/bdr/63c890eb-cd66-4cb5-8518-45bd6dc1409c/occurrence/6167022> AS ?occ)
  
  ?obs
      sosa:hasFeatureOfInterest ?occ ;
      sosa:observedProperty <http://linked.data.gov.au/def/tern-cv/56195246-ec5d-4050-a1c6-af786fbec715> ;
      sosa:hasResult ?r ;
  .
  
  ?r rdf:value ?l
}
```

## 5. Desktop GIS SPARQL Use

So far we've only put queries to files or remote DB's using Python's RDFLib and to DB's by online, web page, SPARQL UIs.

Other software can act as SPARQL clients, for example QGIS, via a plugin.

[QGIS](https://qgis.org/) is a free and open-source desktop GIS environment, similar to ArcGIS Pro.

The [SPARQLing Unicorn Plugin](https://plugins.qgis.org/plugins/sparqlunicorn/) is a QGIS Plugin that can create a "GeoJSON layer from SPARQL endpoint queries".

### 5.1 Connecting to an endpoint

Try using the endpoint `https://api.idnau.org/sparql`. Hopefully, soon, we can use the BDR with this plugin.

* Install it
* Load the plugin via the menu like this: Vector > SPARQL Unicorn Wikidata Plugin > Add GeoJSON Layer from a Wikidata

You will see a window like this appear:

<div>
    <img src="module-5i-files/qgis-endpoint-result.png" width="75%"/>
</div>

Add a new SPARQL endpoint via the "Quick Add RDF Resource" button and see this popup:

<div>
    <img src="module-5i-files/qgis-add-endpoint.png" width="50%"/>
</div>

Add the values as shown in the image above and it will think for a while then report:

<div>
    <img src="module-5i-files/qgis-add-endpoint-detection.png" width="50%"/>
</div>

Don't worry about the error message. The Plugin has detected the SPARQL endpoint and it's GeoSPARQL capabilities.

Select "Add Triple Store" and you will return to this screen:

<div>
    <img src="module-5i-files/qgis-endpoint-result.png" width="75%"/>
</div>

Remove the line `?item <http://www.w3.org/2004/02/skos/core#hasTopConcept> <> .` from the auto-loaded query.

Click the button "add layer".

You will then see something like the following in the main QGIS window:

<div>
    <img src="module-5i-files/qgis-layer.png" width="75%"/>
</div>

The auto-generated queries the plugin made can be altered to get labels and so on from the RDF DB - a lesson for another day!
